# Raport

# Krótkie odpowiedzi na pytania

* Ile znajduje się w zbiorze cech kategorycznych, a ile numerycznych

Mamy 5 wartości numerycznych, pozostałe są nominalne

* Czy zmienna wyjściowa jest kategoryczna, czy numeryczna

Jest numeryczną

* Czy i ile w zbiorze jest brakujących wartości? Dla jakich zmiennych? Co z tego wynika?

Dla wartości niezbędnych do analizy wydźwięku procent brakujących wartości wynosi mniej niż 1 procent

* Czy któreś z cech są skorelowane? Co z tego może wynikać?

Ogólnie nie mamy silnych korelacji

* Czy któraś z cech koreluje ze zmienną wyjściową? Jeśli tak - która? Czy któraś nie koreluje?
'overall' ma relatywnie wysoką korelacje dla czasu i kategorji, ale nie ma dostatecznie silnych korelacji

* Czy zbiór danych wydaje się być wystarczająco informacyjny by rozwiązać zadanie analizy sentymentu?

Tak

* Czy któreś ze słów wydają się dominować w zbiorze?
W tekście dominują słowa "pomocnicze" jak "the", "in", "for" etc. Po usunęciu "common language" dominują słowy opisujący towar (co to jest, czy pracuje)

* Czy najpopularniejsze słowa różnią się znacząco pomiędzy klasami? Czy potrafisz wyróżnić słowa mogące wpływać w znaczym stopniu na sentyment?

Możemy rozróżnić słowy pomiędzy klasami z uwagą, że różnice są mniej widoczne porównując 1 z 2 i 3 z 4

* Jaka jest charakterystyka tekstu 

Najwięcej tekstów o krótkiej długości (do ok. 5 tyś symboli dla 'reviewText' i 60 symboli do 'summary')

# Ogólne wnioski

Dane mają dużo atrybutów, ale do budowania modelu przydatna jest tylko część. Niezbędnymi są atrybuty tekstowe 'summary' i 'reviewText' i atrybut klasy 'overall'.

Do budowaniu modelu mogą też pszydać 'category' i 'unixReviewTime' (lub inna wartość czasu), które nie mają brakujące wartości, jednak to zależy od celu modelu, czy chcemy ogólnie analizować tekst na sentyment lub chcemy model analizy sentymentów dla review na Amazon dla wybranych kategorii i/lub czasu (bo dla różnych kategorii i czasu mogą być inne kryterium dla dobrego lub złego produktu).

Nie mamy silnych korelacji pomiędzy atrybutami numerycznymi. Ale nie jest to problemem bo bardziej nam potrzebne wartości nominalne i nie stoi nam problem redukcji wymiarowości.

Jedną z największych problemów który ma ten dataset to nierównomierny rozkład wartości klas. Klas o wartości '1' (najlepszy sentyment) jest więcej niż wszystkich pozostałych. Może to spowodować problemy z uczeniem i ewaluacją modelu, bo model może się nauczyć wykrywać tylko klasę '1'.

Następnym problemem jest jakie słowa używać, bo najwięcej występują słowy zawierające niewiele informacji jak "the", "for", "i" itd. Możemy przy pomocy bibliotek usunąć takie słowy, ale wtedy tracimy takie słowy jak "not" i "but", które zawirają potzebną nam informacje. Ale nawet po usunięciu "common language" mamy zbędne słowy jako typ produktu lub jakieś przedmioty. Dlatego pytaniem jest jak wybrać potrzebne lub jak usunąć niepotrzebne słowy dla budowania modelu, czy wybrać pewne słowy które będą używane do analizy lub dopasować listę zbędnych słów aby nie usuwały się nam słowy niezbędne.



# Początek EDA

Analiza danych po ekstrakcji cech

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import collections
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

pd.options.display.max_columns = 30

In [ ]:
split_path = './data/features/'

train_file_name = 'Amazon_train.json'
test_file_name = 'Amazon_test.json'

df_train = pd.read_json(os.path.join(split_path, train_file_name))
df_test = pd.read_json(os.path.join(split_path, test_file_name))

df = pd.concat([df_train, df_test], axis = 0)

# Ogólna analiza danych

In [ ]:
df

In [ ]:
(df.isna().sum(axis=0).to_frame().transpose() / df.shape[0] * 100)


* Czy i ile w zbiorze jest brakujących wartości? Dla jakich zmiennych? Co z tego wynika?

W polach 'summary' i 'reviewText', zawierające tekst i które mają największe znaczenie w analizie wydźwięku, brakujących wartości jest mniej 1%, co jest dobre, bo prawie całe dane są przygodne do analizy.

Nie mamy brakujących wartości w etykietach klas, co też jest dobre.

Dużo brakujących wartości mamy w 'style_list' (oryginalnie Style) i odpowiednio w atrybutach, które powstały w procesie atomizacji: 'package quantity', 'design', 'format', 'scent name', 'flavor', 'style', 'package type', 'style name', 'color, size', 'size name', 'platform'. Nie są do użycia w trenowaniu modelu.

Mamy dużo brakujących wartości w vote, to jest liczba odznaczeń od innych użytkowników, że ten review jest użyteczny. Nie obchodzi nas w analizie wydźwięku 

Image ma dużo brakujących wartości, ale ona zawiera link do obrazu, nie obchodzi nas w analizie wydźwięku

'reviewerID' to ID użytkownika, nie ma brakujących wartości

'asin' to ID produktu na Amazon, nie ma brakujących wartości

'category' jest to kategoria produktu, nie ma brakujących wartości

'reviewTime' i 'unixReviewTime' to czas (data w godzinie 00:00) w którym review został wysłany, nie ma brakujących wartości

'summary_length', 'year_period' i 'time_period' to atrybuty oparte na atrybutach 'summary' i 'unixReviewTime' (i odpowiednio reviewTime)

'year_period' i 'time_period' to wartości kategoryczne

* Czy zbiór danych wydaje się być wystarczająco informacyjny by rozwiązać zadanie analizy sentymentu?

W najważniejszych atrybutach dla analizy sentymentu, 'overall', 'summary' i 'reviewText', mamy bardzo mało brakujących wartości, dlatego prawie cały dataset jest przydatny do analizy. 


# Analiza danych numerycznych

In [ ]:
df.describe()

* Ile znajduje się w zbiorze cech kategorycznych, a ile numerycznych

W zbiorze mamy 5 wartości numerycznych

* Czy zmienna wyjściowa jest kategoryczna, czy numeryczna? 

Etykieta klasy jest wartością numeryczną, co znaczy że output modelu jest też numeryczny

In [ ]:
df_corr = df[['overall', 'unixReviewTime', 'summary_length', 'reviewText_length', 'category', 'year_period']]

# 'package quantity' usunięte bo maje tylko jedną wartość

df_corr['category'] = df_corr['category'].apply(lambda x: 1 if x == 'Software_5' 
                                                else 2 if x =='All_Beauty_5' 
                                                else 3 if x =='AMAZON_FASHION_5'
                                                else 4)

df_corr['year_period'] = df_corr['year_period'].apply(lambda x: 1 if x =='winter' 
                                                else 2 if x =='spring' 
                                                else 3 if x =='autumn'
                                                else 4)

df_corr.corr()

* Czy któreś z cech są skorelowane? Co z tego może wynikać?

Mamy jakoś korelacje pomiędzy 'unixReviewTime' i 'category', to może wskazać na sezonowe trendy

Mamy małą korelacje pomiędzy 'summary_length' i 'reviewText_length', długie teksty mają długie uogólnienia

Mamy mała korelacje pomiędzy 'category' i 'overall', możemy zrobić hipotezę że jakieś kategorie produktów ogólnie otrzymują 

Ogólnie nie mamy silnych korelacji

* Czy któraś z cech koreluje ze zmienną wyjściową? Jeśli tak - która? Czy któraś nie koreluje?

'overall' ma relatywnie wysoką korelacje dla czasu i kategorii, ale nie ma dostatecznie silnych korelacji

# Analiza danych tekstowych

In [ ]:
all_words_inReview = [y.lower() for x in df['reviewText'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if len(y) > 0]
all_words_inSummary = [y.lower() for x in df['summary'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if len(y) > 0]

words_counter_Rev = collections.Counter(all_words_inReview)
words_counter_Sum = collections.Counter(all_words_inSummary)

Top N najwięcej występujących słów

In [ ]:
N = 15

print('\nSłowy w reviewText')
for x in sorted(list(words_counter_Rev.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nSłowy w summary')
for x in sorted(list(words_counter_Sum.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nOverall')
for x in sorted(list((words_counter_Rev + words_counter_Sum).items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)



* Czy któreś ze słów wydają się dominować w zbiorze?

W reviewText najwięcej występują słowy 'wspomagające' i nie niosą dużo informacji. Taka sama tendencja jest i w Summary.

Ciekawą informacje mamy w 'summary', najwięcej mamy słów pozytywne "five" "stars" "good". Aby zrobić z tego wniosek musimy sprawdzić występowanie klas.

In [ ]:
df['overall'].value_counts()

To potwierdza nasze obserwacje w 'summary', najwięcej mamy pozytywnych ocen, ocen w 5 gwiazd jest więcej niż wszystkich pozostałych ocen.

$$---$$

Teraz użyjemy NLTK aby usunąć 'common language'

In [ ]:
all_words_inReview = [y.lower() for x in df['reviewText'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if (len(y) > 0)]
all_words_inSummary = [y.lower() for x in df['summary'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if (len(y) > 0)]

words_counter_Rev = collections.Counter((x,y) for x,y in collections.Counter(all_words_inReview).items() if not x in stopwords.words())
words_counter_Sum = collections.Counter((x,y) for x,y in collections.Counter(all_words_inSummary).items() if not x in stopwords.words())


In [ ]:
N = 15

print('\nSłowy w reviewText')
for x in sorted(list(words_counter_Rev.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nSłowy w summary')
for x in sorted(list(words_counter_Sum.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nOverall')
for x in sorted(list((words_counter_Rev + words_counter_Sum).items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)



Widać że w 'reviewText' najwięcej słów opisujących produkt i w wybranych słowach tylko "great" jest słowem opisującym emocje.

To samo mniej więcej widać w 'summary', ale mamy więcej słów oceniających

* Jaka jest charakterystyka tekstu (np. długość, czystość)?

In [ ]:
plt.yscale("log")
plt.xlabel('reviewText_length')
plt.hist(df['reviewText_length'])
plt.show()

In [ ]:
plt.yscale("log")
plt.xlabel('summary_length')
plt.hist(df['summary_length'])

plt.show()

# Analiza tekstu class-wise

In [ ]:
classNum = 5
N = 15

all_words_inReview = [y.lower() for x in df[df['overall'] == classNum]['reviewText'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if len(y) > 0]
all_words_inSummary = [y.lower() for x in df[df['overall'] == classNum]['summary'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if len(y) > 0]

words_counter_Rev = collections.Counter(all_words_inReview)
words_counter_Sum = collections.Counter(all_words_inSummary)

print('\nSłowy w reviewText')
for x in sorted(list(words_counter_Rev.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nSłowy w summary')
for x in sorted(list(words_counter_Sum.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nOverall')
for x in sorted(list((words_counter_Rev + words_counter_Sum).items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

* Czy najpopularniejsze słowa różnią się znacząco pomiędzy klasami? Czy potrafisz wyróżnić słowa mogące wpływać w znacznym stopniu na sentyment?

Po analizie co klasy możemy wyróżnić następne słowy (dla wszystkich kategorii słów):

1: not, buy, don't

2: not

3: not, but, good, needs

4: good, great, works, but, four, just

5: five, great, excellent, love,

$$---$$

Analiza z NLTK

In [ ]:
classNum = 1
N = 15

all_words_inReview = [y.lower() for x in df[df['overall'] == classNum]['reviewText'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if len(y) > 0]
all_words_inSummary = [y.lower() for x in df[df['overall'] == classNum]['summary'].to_list() if x != None for y in re.split('\s|!|,|\.|\?', x)  if len(y) > 0]

words_counter_Rev = collections.Counter(dict((x,y) for x,y in collections.Counter(all_words_inReview).items() if not x in stopwords.words()))
words_counter_Sum = collections.Counter(dict((x,y) for x,y in collections.Counter(all_words_inSummary).items() if not x in stopwords.words()))

print('\nSłowy w reviewText')
for x in sorted(list(words_counter_Rev.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nSłowy w summary')
for x in sorted(list(words_counter_Sum.items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

print('\nOverall')
for x in sorted(list((words_counter_Rev + words_counter_Sum).items()), key = lambda x: x[1], reverse = True)[:N]:
    print('\t',x)

Po analizie co klasy możemy wyróżnić następne słowy (dla wszystkich kategorii słów):

1: failures, trouble, incompatibilities, waste, money, worthless, painfully

2: buggy, free, problem, poor, slow

3: easy, great, works, decent

4: easy, great, excellent, nice, works

5: great, love, easy, clean, easy, excellent, woked, works